Import relevant libraries

In [ ]:
import pandas as pd
from Preprocessing.preprocessing import preprocessing
import time
import torch
from tsai.all import *

print('GPU support:', torch.cuda.is_available())
computer_setup()

c:\Users\karll\miniconda3\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


GPU support: False
os              : Windows-10-10.0.22621-SP0
python          : 3.9.13
tsai            : 0.3.5
fastai          : 2.7.11
fastcore        : 1.5.28
torch           : 1.13.1+cpu
cpu cores       : 6
threads per cpu : 2
RAM             : 15.9 GB
GPU memory      : [6.0] GB


In [ ]:
from Optimizer.optimizer import optimize_model

df = pd.read_csv('Data\Stock\StockBars\MSFT_Minute')
 
preprocessing_params = {
    'df': df[:1000],
    'lag': 1,
    'dif_all': True,
    'train_size': 0.8,
    'TSAI': True,
    'CLF': True,
    'index': None,
    'data': "alpacca",
    'buckets': 1
}

model_type = 'tst_class'

opti = True
if opti:
    optimize_model(model_type=model_type, preprocessing_params=preprocessing_params, n_trials=2)

results_df = pd.read_csv(f"models/{model_type}/{model_type}_hyperparameters_results.csv")
results_df

In [ ]:
from Optimizer.optimizer import optimize_data_classification, optimize_data_regression

df = pd.read_csv('Data\\twelve_data\AAPL_1min')[:2000]
optimize_data_classification(df,'twelve','1min',2,5,5)


[I 2023-03-22 10:44:33,784] A new study created in memory with name: no-name-9ae5c0ab-10f0-4685-bd68-08b38c3d00ec


               datetime       open       high        low      close  volume  \
23  2021-02-16 12:49:00  133.52499  133.52991  133.31000  133.37849  183270   
24  2021-02-16 12:50:00  133.38000  133.41000  133.31000  133.40759   95343   
25  2021-02-16 12:51:00  133.39500  133.42999  133.36000  133.36501   77483   
26  2021-02-16 12:52:00  133.36000  133.42999  133.36000  133.42990  125363   
27  2021-02-16 12:53:00  133.42500  133.49500  133.42500  133.49500   95862   
28  2021-02-16 12:54:00  133.49500  133.50000  133.46730  133.47501  105994   
29  2021-02-16 12:55:00  133.48000  133.48080  133.44000  133.46001   77085   
30  2021-02-16 12:56:00  133.47000  133.50000  133.44930  133.44930   86462   
31  2021-02-16 12:57:00  133.44000  133.46989  133.41000  133.42000   72956   
32  2021-02-16 12:58:00  133.41150  133.53000  133.41000  133.50999   63934   
33  2021-02-16 12:59:00  133.50999  133.53999  133.47501  133.47501   63385   
34  2021-02-16 13:00:00  133.47501  133.50079  133.4

[W 2023-03-22 10:44:35,279] Trial 0 failed with parameters: {'seq_length': 26, 'lag': 3, 'dif_all': False, 'TI': True, 'index': None, 'batch_size': 128} because of the following error: IndexError('Target -9223372036854775808 is out of bounds.').
Traceback (most recent call last):
  File "c:\Users\karll\miniconda3\lib\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "c:\Users\karll\Documents\Data science\Examensarbete\Master-Thesis\Optimizer\optimizer.py", line 224, in objective
    learn.fit_one_cycle(epochs, lr_max=0.001)
  File "c:\Users\karll\miniconda3\lib\site-packages\fastai\callback\schedule.py", line 119, in fit_one_cycle
    self.fit(n_epoch, cbs=ParamScheduler(scheds)+L(cbs), reset_opt=reset_opt, wd=wd, start_epoch=start_epoch)
  File "c:\Users\karll\miniconda3\lib\site-packages\fastai\learner.py", line 264, in fit
    self._with_events(self._do_fit, 'fit', CancelFitException, self._end_cleanup)
  File "c:\Users\karll\

IndexError: Target -9223372036854775808 is out of bounds.

In [ ]:
model_type = 'lstm_class'

with open(f"models/{model_type}/{model_type}_best_params.json", "r") as f:
    best_params = json.load(f)

best_params

{'seq_length': 200,
 'batch_size': 16,
 'hidden_size': 25,
 'n_layers': 1,
 'rnn_dropout': 0.5,
 'fc_dropout': 0.4,
 'learning_rate': 0.0019302736528178678}

Select a stock for Machine Learning Model Training and preprocesses it

In [ ]:
# Preprocess the data using a custom function and split it into training and testing sets
# Only the training and testing sets are used, so the third variable (a scalar) is discarded using an underscore

seq_length = best_params.pop('seq_length', None)
data_train, data_test, _ = preprocessing(**preprocessing_params, sequence_length=seq_length, print_info=True)

# Changes the data into features and labels with the split used later in TSAI for modelling
X, y, splits = combine_split_data([data_train[0], data_test[0]],[data_train[1], data_test[1]])

# Utilizes the GPU if possible
if torch.cuda.is_available(): X, y = X.cuda(), y.cuda()

batch_size = best_params.pop('batch_size', None)
dsets = TSDatasets(X, y, splits=splits)
dls = TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=batch_size)

# Note this tabel is before sequenceing so the actuall values is total - sequence then times train_size

   Label  Count  Train count  Test count  Bucket min  Bucket max
0    0.0    432          338          93     -1.0488     -0.0001
1    1.0    566          460         106      0.0000      0.9588
2  Total    998          798         199     -1.0488      0.9588


Initializes the models and learners

In [ ]:
nr_features = X.shape[1] # Number of features
nr_labels = torch.unique(y).numel() # Number of labels

model_params = {key: value for key, value in best_params.items() if key not in ('seq_length', 'batch_size', 'learning_rate')}

# Initiates the models
model_lstm_fcn = LSTM_FCNPlus(c_in=nr_features, c_out=nr_labels, **model_params, shuffle=False)
#model_lstm = LSTMPlus(c_in=nr_features, c_out=nr_labels, seq_len=sequence_length)
#model_tst = TST(c_in=nr_features, c_out=nr_labels, seq_len=sequence_length)
#model_xcm = XCMPlus(c_in=nr_features, c_out=nr_labels, seq_len=sequence_length)

models = {'LSTM_FCNPlus': model_lstm_fcn}#, 'LSTMPlus': model_lstm, 'TST': model_tst, 'XCMPlus': model_xcm}

# Create Learner objects
binary_classification_metrics = [accuracy]

learn_lstm_fcn = Learner(dls, model_lstm_fcn, loss_func=LabelSmoothingCrossEntropyFlat(), metrics=accuracy)
#learn_lstm = Learner(dls, model_lstm, loss_func=LabelSmoothingCrossEntropyFlat())
#learn_tst = Learner(dls, model_tst, loss_func=LabelSmoothingCrossEntropyFlat())
#learn_xcm = Learner(dls, model_xcm, loss_func=LabelSmoothingCrossEntropyFlat())

learners = {'LSTM_FCNPlus': learn_lstm_fcn}#, 'LSTMPlus': learn_lstm, 'TST': learn_tst, 'XCMPlus': learn_xcm}


TypeError: __init__() got an unexpected keyword argument 'n_layers'

Find optimal learning rates

In [ ]:
results = pd.DataFrame(columns=['Model Name', 'Rate', 'Accuracy', 'Training Time'])

epochs = 3

learning_rate = best_params.pop('learning_rate', None)


for name, learner in learners.items():
    start_time = time.time()
    learner.fit_one_cycle(n_epoch=epochs, lr_max=learning_rate)
    end_time = time.time()
    training_time = round(end_time - start_time, 2)

    loss = learner.validate()

loss

(#2) [0.7115873694419861,0.5]